# Draft Model

In [2]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import requests

In [3]:
ADP_URL = "https://www.fantasypros.com/nfl/adp/ppr-overall.php"

In [4]:
def make_adp_df():
    res = requests.get(ADP_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id':'data'})
        df = pd.read_html(str(table))[0]
        print('Output after reading the html:\n\n',df.head(),'\n')
        df = df[['Player Team (Bye)','POS','AVG']]
        print('Output after filtering:\n\n',df.head(),'\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x:' '.join(x.split()[:-2]))
        df['POS'] = df['POS'].apply(lambda x:x[:2])
        
        df = df[['PLAYER','POS','AVG']].sort_values(by='AVG')
        
        print('Final output:\n\n', df.head())
        
        return df
    
    else:
        print("Didn't work, try again")

In [5]:
df = make_adp_df()

Output after reading the html:

    Rank             Player Team (Bye)  POS  ESPN  MFL  RTSports  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN       1.0      1.0  1.0
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN       2.0      2.0  2.0
2     3      Ezekiel Elliott DAL (10)  RB3   3.0  NaN       3.0      3.0  3.0
3     4           Dalvin Cook MIN (7)  RB4   4.0  NaN       4.0      6.0  4.7
4     5         Michael Thomas NO (6)  WR1   6.0  NaN       5.0      4.0  5.0 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.0
3           Dalvin Cook MIN (7)  RB4  4.7
4         Michael Thomas NO (6)  WR1  5.0 

Final output:

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1       Saquon Barkley  RB  2.0
2      Ezekiel Elliott  RB  3.0
3          Dalvin Cook  RB  4.7
4       Michael Thomas  WR  5.0


In [6]:
df.head()

,PLAYER,POS,AVG
0,Christian McCaffrey,RB,1.0
1,Saquon Barkley,RB,2.0
2,Ezekiel Elliott,RB,3.0
3,Dalvin Cook,RB,4.7
4,Michael Thomas,WR,5.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596 entries, 0 to 595
Data columns (total 3 columns):
PLAYER    596 non-null object
POS       596 non-null object
AVG       596 non-null float64
dtypes: float64(1), object(2)
memory usage: 18.6+ KB


In [8]:
replacement_players = {
    'RB':None,
    'WR':None,
    'TE':None,
    'QB':None
}

In [9]:
replacement_players

{'RB': None, 'WR': None, 'TE': None, 'QB': None}

In [10]:
# look at the top 100 picks in our df and get the last RB, WR, QB, and TE
# and store them in our dict

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player
    
replacement_players

{'RB': 'Matt Breida',
 'WR': 'Sterling Shepard',
 'TE': 'Jared Cook',
 'QB': 'Carson Wentz'}

Get projected player scores

In [11]:
BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'

def make_projection_df():
    
    final_df = pd.DataFrame()
    
    for position in ['rb', 'qb', 'te', 'wr']:
        
        res = requests.get(BASE_URL.format(position=position))
        if res.ok:
            soup = BS(res.content, 'html.parser')
            table = soup.find('table', {'id': 'data'})
            df = pd.read_html(str(table))[0]
            
            df.columns = df.columns.droplevel(level=0)
            df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1]))
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + df['REC']
                
            df['POS'] = position.upper()
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            final_df = pd.concat([final_df, df])
            
        else:
            print("Didn't work, try again")
            return
    
    final_df = final_df.sort_values(by='FPTS', ascending=False)
    
    return final_df

In [12]:
df = make_projection_df()
df.head()

,PLAYER,POS,FPTS
0,Christian McCaffrey,RB,375.0
0,Lamar Jackson,QB,361.1
1,Patrick Mahomes,QB,345.0
0,Michael Thomas,WR,333.1
2,Dak Prescott,QB,318.4


Calculate replacement values

In [13]:
replacement_values = {
    'RB':None,
    'WR':None,
    'TE':None,
    'QB':None
}

replacement_values

{'RB': None, 'WR': None, 'TE': None, 'QB': None}

In [14]:
for position, player in replacement_players.items():
    if position in ['QB','TE', 'WR', 'RB']:
        replacement_values[position] = df.loc[df['PLAYER'] == player].values[0,-1]
        
replacement_values

{'RB': 139.0, 'WR': 183.8, 'TE': 146.2, 'QB': 274.5}

In [15]:
df['VOR'] = df.apply(
    lambda row: row['FPTS'] - replacement_values.get(row['POS']), 
    axis=1
)
df.head()

,PLAYER,POS,FPTS,VOR
0,Christian McCaffrey,RB,375.0,236.0
0,Lamar Jackson,QB,361.1,86.6
1,Patrick Mahomes,QB,345.0,70.5
0,Michael Thomas,WR,333.1,149.3
2,Dak Prescott,QB,318.4,43.9


In [16]:
df = df.sort_values(by='VOR', ascending=False)
df['VALUERANK'] = df['VOR'].rank(ascending=False)
df.head(50)

,PLAYER,POS,FPTS,VOR,VALUERANK
0,Christian McCaffrey,RB,375.0,236.0,1.0
1,Saquon Barkley,RB,310.7,171.7,2.0
2,Ezekiel Elliott,RB,299.6,160.6,3.0
5,Alvin Kamara,RB,296.2,157.2,4.0
4,Dalvin Cook,RB,289.9,150.9,5.0
0,Michael Thomas,WR,333.1,149.3,6.0
3,Derrick Henry,RB,261.4,122.4,7.0
6,Clyde Edwards-Helaire,RB,254.1,115.1,8.0
1,Davante Adams,WR,298.8,115.0,9.0
13,Austin Ekeler,RB,252.6,113.6,10.0


In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [17]:
# fig = plt.figure(figsize=(10,8))

# sns.lineplot(x='VALUERANK', y='VOR', data=df[df['POS'] == 'RB'])

# plt.show()

In [18]:
positions = list(set(df.POS.values))
positions

['TE', 'QB', 'RB', 'WR']

In [19]:
# for p in positions:
#     sns.lineplot(x='VALUERANK', y='VOR', data=df[df['POS'] == p])
#     plt.title(p)
#     plt.show()

# Avg Draft Pick

In [19]:
adp_df = make_adp_df()
adp_df.head()

Output after reading the html:

    Rank             Player Team (Bye)  POS  ESPN  MFL  RTSports  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN       1.0      1.0  1.0
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN       2.0      2.0  2.0
2     3      Ezekiel Elliott DAL (10)  RB3   3.0  NaN       3.0      3.0  3.0
3     4           Dalvin Cook MIN (7)  RB4   4.0  NaN       4.0      6.0  4.7
4     5         Michael Thomas NO (6)  WR1   6.0  NaN       5.0      4.0  5.0 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.0
3           Dalvin Cook MIN (7)  RB4  4.7
4         Michael Thomas NO (6)  WR1  5.0 

Final output:

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1       Saquon Barkley  RB  2.0
2      Ezekiel Elliott  RB  3.0
3          Dalvin Cook  RB  4.7
4       Michael Thomas  WR  5.0


,PLAYER,POS,AVG
0,Christian McCaffrey,RB,1.0
1,Saquon Barkley,RB,2.0
2,Ezekiel Elliott,RB,3.0
3,Dalvin Cook,RB,4.7
4,Michael Thomas,WR,5.0


In [20]:
adp_df['ADPRANK'] = adp_df['AVG'].rank(method='first')

adp_df.head()

,PLAYER,POS,AVG,ADPRANK
0,Christian McCaffrey,RB,1.0,1.0
1,Saquon Barkley,RB,2.0,2.0
2,Ezekiel Elliott,RB,3.0,3.0
3,Dalvin Cook,RB,4.7,4.0
4,Michael Thomas,WR,5.0,5.0


Merge dataframes

In [21]:
df = df.merge(adp_df, how='left', on=['PLAYER', 'POS'])

df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0
2,Ezekiel Elliott,RB,299.6,160.6,3.0,3.0,3.0
3,Alvin Kamara,RB,296.2,157.2,4.0,5.3,6.0
4,Dalvin Cook,RB,289.9,150.9,5.0,4.7,4.0


In [22]:
df.head(10)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0
2,Ezekiel Elliott,RB,299.6,160.6,3.0,3.0,3.0
3,Alvin Kamara,RB,296.2,157.2,4.0,5.3,6.0
4,Dalvin Cook,RB,289.9,150.9,5.0,4.7,4.0
5,Michael Thomas,WR,333.1,149.3,6.0,5.0,5.0
6,Derrick Henry,RB,261.4,122.4,7.0,7.3,7.0
7,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0
8,Davante Adams,WR,298.8,115.0,9.0,11.3,8.0
9,Austin Ekeler,RB,252.6,113.6,10.0,15.0,14.0


In [24]:
df['SLEEPERSCORE'] = df['ADPRANK'] - df['VALUERANK']

df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,0.0
2,Ezekiel Elliott,RB,299.6,160.6,3.0,3.0,3.0,0.0
3,Alvin Kamara,RB,296.2,157.2,4.0,5.3,6.0,2.0
4,Dalvin Cook,RB,289.9,150.9,5.0,4.7,4.0,-1.0


# Sleeper Score 

In [25]:
df.loc[df['AVG'] < 160].sort_values(by='SLEEPERSCORE', ascending=False).head(20)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
42,Antonio Gibson,RB,182.9,43.9,42.5,136.0,133.0,90.5
114,Jonnu Smith,TE,130.1,-16.1,115.0,159.0,159.0,44.0
110,Dallas Goedert,TE,132.3,-13.9,110.5,148.3,147.0,36.5
49,Tarik Cohen,RB,174.8,35.8,50.0,87.3,85.0,35.0
121,Jack Doyle,TE,126.6,-19.6,122.0,156.7,156.0,34.0
94,Mike Gesicki,TE,146.0,-0.2,95.0,131.0,127.0,32.0
115,Ryan Tannehill,QB,258.2,-16.3,116.0,147.0,146.0,30.0
100,Duke Johnson,RB,134.5,-4.5,101.0,132.3,130.0,29.0
109,Jared Goff,QB,260.6,-13.9,110.5,139.7,137.0,26.5
118,DeSean Jackson,WR,165.4,-18.4,119.0,146.3,145.0,26.0


In [26]:
df.loc[df['AVG'] < 160].sort_values(by='SLEEPERSCORE', ascending=True).head(20)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
330,Ke'Shawn Vaughn,RB,17.6,-121.4,331.0,144.3,141.0,-190.0
211,Alshon Jeffery,WR,106.3,-77.5,212.0,149.0,149.0,-63.0
191,Mecole Hardman,WR,123.0,-60.8,192.0,136.0,134.0,-58.0
162,Alexander Mattison,RB,94.5,-44.5,163.5,126.3,122.0,-41.5
120,Deebo Samuel,WR,164.9,-18.9,121.0,86.3,83.0,-38.0
155,Justin Jefferson,WR,143.9,-39.9,156.0,120.7,118.0,-38.0
154,Darrell Henderson,RB,99.2,-39.8,155.0,124.7,120.0,-35.0
147,Jerry Jeudy,WR,147.9,-35.9,148.0,115.3,114.0,-34.0
123,J.K. Dobbins,RB,119.1,-19.9,124.0,90.7,90.0,-34.0
78,Leonard Fournette,RB,149.8,10.8,79.0,47.3,47.0,-32.0


# Keeper Analysis

In [27]:
keeper_df = pd.read_excel('Keeper_Master Document.xlsx',sheet_name=None)

In [29]:
keeper_df.keys()

odict_keys(['Keeper History', '2020', '2019', '2018', '2017', '2016 ', '2015'])

In [30]:
keeper_df['Keeper History'].head()

,Year,Team,Player,Round,Position,Notes
0,2015,David,Adrian Peterson,1,NaN,NaN
1,2015,David,Randall Cobb,3,NaN,NaN
2,2015,David,Alshon Jeffrey,13,NaN,NaN
3,2015,Dinatalie,Eddie Lacy,4,NaN,NaN
4,2015,Dinatalie,Jordan Matthews,17,NaN,NaN


In [31]:
keeper_history = keeper_df['Keeper History']
keeper_history.head()

,Year,Team,Player,Round,Position,Notes
0,2015,David,Adrian Peterson,1,NaN,NaN
1,2015,David,Randall Cobb,3,NaN,NaN
2,2015,David,Alshon Jeffrey,13,NaN,NaN
3,2015,Dinatalie,Eddie Lacy,4,NaN,NaN
4,2015,Dinatalie,Jordan Matthews,17,NaN,NaN


In [32]:
keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [33]:
keepers_2020

array(['Davante Adams', 'George Kittle', 'DJ Moore', 'DJ Chark',
       'Juju Smith Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'DK Metcalf',
       'Brashad Perrimen', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'AJ Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliot ',
       'Nick Chubb', 'Devante Parker ', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a '], dtype=object)

In [34]:
len(df.PLAYER.values)

608

# Name Replacements

In [35]:
name_replacements = [('DJ Moore', 'D.J. Moore'),
 ('DJ Chark', 'D.J. Chark'),
 ('Juju Smith Schuster', 'JuJu Smith-Schuster'),
 ('JuJu Smith', 'JuJu Smith-Schuster'),
 ('DK Metcalf', 'D.K. Metcalf'),
 ('Brashad Perrimen', 'Breshad Perriman'),
 ('AJ Brown', 'A.J. Brown'),
 ('Ronald Jones', 'Ronald Jones II'),
 ('Ezekiel Elliot ', 'Ezekiel Elliott'),
 ('Ezekiel Elliot', 'Ezekiel Elliott'),
 ('Devante Parker ', 'DeVante Parker'),
 ('Devante Parker', 'DeVante Parker'),
 ('Saquon Barkely', 'Saquon Barkley'),
 ('Christian McAffrey', 'Christian McCaffrey'),
 ('Adam Thielan', 'Adam Thielen'),
 ('LeVeon Bell', "Le'Veon Bell"),
 ('Leornard Fournette', 'Leonard Fournette'),
 ('Mark Ingram', 'Mark Ingram II'),
 ('Mark Ingram ', 'Mark Ingram II'),
 ('TY Hilton', 'T.Y. Hilton'),
 ('Derius Guice', 'Derrius Guice'),
 ('AJ Green', 'A.J. Green'),
 ('Odell Beckham', 'Odell Beckham Jr.'),
 ('L McCoy', 'LeSean McCoy'),
 ('Alshon Jeffrey', 'Alshon Jeffery'),
 ('Marvin Jones Jr', 'Marvin Jones'),
 ('Darell Henderson', 'Darrell Henderson'),
 ('Russel Wilson', 'Russell Wilson'),
 ('A Miller', 'Anthony Miller'),
 ('DJ Metcalf', 'D.K. Metcalf'),
 ('Desean Jackson', 'DeSean Jackson'),
 ('Gio Bernard', 'Giovani Bernard'),
 ('Justice Mill', 'Justice Hill'),
 ('Kennan Ballage', 'Kalen Ballage'),
 ('Malcom Brown', 'Malcolm Brown'),
 ('Marques Valdez Scandling', 'Marquez Valdes-Scantling'),
 ('OJ Howard', 'O.J. Howard'),
 ('Philip Lindsay', 'Phillip Lindsay'),
 ('R Higgins', 'Rashard Higgins'),
 ('Rashad Penny', 'Rashaad Penny'),
 ('Tyreke Hill', 'Tyreek Hill'),
 ('TJ Hockenson', 'T.J. Hockenson')]

In [36]:
kickers = ['Greg Zuerlin',
 'Harrison Butker',
 'Justin Tucker',
 'Kai Fairbarn',
 'Mike Badgley',
 'Robbie Gould',
 'Stephen Gostowski',
 'Will Lutz',
 'Younghoe Koo',
 'Wil Lutz',
 'Austin Seibert']

In [37]:
defenses = ['Bears',
 'Rams',
 'Browns',
 'Eagles',
 'Jaguars',
 'Patriots',
 'Ravens',
 'Vikings',
 '49ers',
 'Broncos',
 'Chiefs',
 'Falcons',
 'Jets',
 'Pats']

In [38]:
unsigned_fa = ['Devonta Freeman',
 'Antonio Brown',
 'Derrius Guice',
 'Damien Williams',
 'Delanie Walker',
 'Geronimo Allison',
 'Josh Gordon',
 'Taylor Gabriel',
 'Tyrell Williams',
 'Adrian Peterson']

## Cleaning Keeper List

In [39]:
draft_19 = keeper_df['2019']

In [40]:
def check_names(check_list, correct_list):
    l = []
    for p in check_list:
        if p not in correct_list:
            l.append(p)
    return l

def find_name(name, source=df.PLAYER.values):
    for p in source:
        if name in p:
            print(p)
            
def name_replacer(wrong_name, correct_name, data=draft_19):
    data.replace(to_replace=wrong_name, value=correct_name, inplace=True)
    
def remove_fa_k_dst(main_list, fa, k, dst):
    main_list = np.setdiff1d(main_list, fa)
    main_list = np.setdiff1d(main_list, k)
    main_list = np.setdiff1d(main_list, dst)
    return main_list

In [41]:
keeper_replacements = [
    ('DJ Moore', 'D.J. Moore'), 
    ('DJ Chark', 'D.J. Chark'),
    ('Juju Smith Schuster', 'JuJu Smith-Schuster'),
    ('DK Metcalf', 'D.K. Metcalf'),
    ('Brashad Perrimen', 'Breshad Perriman'),
    ('AJ Brown', 'A.J. Brown'),
    ('Ronald Jones', 'Ronald Jones II'),
    ('Ezekiel Elliot ', 'Ezekiel Elliott'),
    ('Devante Parker ', 'DeVante Parker')
]

In [42]:
for p in keeper_replacements:
    keeper_history.replace(to_replace=p[0], value=p[1], inplace=True)

keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
    
keepers_2020

array(['Davante Adams', 'George Kittle', 'D.J. Moore', 'D.J. Chark',
       'JuJu Smith-Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'D.K. Metcalf',
       'Breshad Perriman', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'A.J. Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones II', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliott',
       'Nick Chubb', 'DeVante Parker', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a '], dtype=object)

In [45]:
names_to_fix = check_names(keepers_2020, df.PLAYER.values)
names_to_fix

['Adrian Peterson', 'N/a ', 'N/a ']

In [48]:
keeper_list = list(keepers_2020)

In [73]:
# keepers cleaned
keepers_2020 = np.array(keeper_list)
keepers_2020

array(['Davante Adams', 'George Kittle', 'D.J. Moore', 'D.J. Chark',
       'JuJu Smith-Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'D.K. Metcalf',
       'Breshad Perriman', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'A.J. Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones II', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliott',
       'Nick Chubb', 'DeVante Parker', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a ', 'Marquise Brown',
       'Russell Wilson', 'Allen Robinson', 'Marlon Mack',
       'Saquon Barkley', 'Miles Sanders', 'David Johnson'], dtype='<U19')

# Last Year's Draft Picks

In [74]:
draft_19 = keeper_df['2019']

In [47]:
draft_19.columns[:8]

Index(['Tony', 'Mike', 'Joey', 'Vlad', 'Dave', 'Chris', 'Matt', 'Ben'], dtype='object')

In [48]:
draft_19.index[:18]

Index(['Round 1', 'Round 2', 'Round 3', 'Round 4', 'Round 5', 'Round 6',
       'Round 7', 'Round 8', 'Round 9', 'Round 10', 'Round 11', 'Round 12',
       'Round 13', 'Round 14', 'Round 15', 'Round 16', 'Round 17', 'Round 18'],
      dtype='object')

In [49]:
draft_19 = draft_19.iloc[:, :8]
draft_19

,Tony,Mike,Joey,Vlad,Dave,Chris,Matt,Ben
Round 1,Saquon Barkely,Ezekiel Elliot,Deshaun Watson,Julio Jones,Todd Gurley,Travis Kelce,Antonio Brown,Christian McAffrey
Round 2,Adam Thielan,Josh Jacobs,Amari Cooper,Chris Carson,Zach Ertz,Dalvin Cook,LeVeon Bell,Keenan Allen
Round 3,David Montgomery,Sony Michel,Leornard Fournette,Mike Williams,Mark Ingram,Melvin Gordon,Derrick Henry,Kenny Golladay
Round 4,Brandin Cooks,Kerryon Johnson,Stefon Diggs,Chris Godwin,DeAndre Hopkins,Julian Edelman,TY Hilton,Derius Guice
Round 5,Tevin Coleman,Mike Evans,Austin Ekeler,Evan Engram,Tyler Boyd,Calvin Ridley,AJ Green,Tyler Lockett
Round 6,Allen Robinson,Robert Woods,Tony Pollard,Devin Singletary,Christian Kirk,Odell Beckham,L McCoy,Miles Sanders
Round 7,Aaron Rodgers,Robby Anderson,Jarvis Landry,Baker Mayfield,Duke Johnson,Alshon Jeffrey,Courtland Sutton,Cam Newton
Round 8,Jordan Howard,Marvin Jones Jr,Sammy Watkins,Darell Henderson,Russel Wilson,Dede Westbrook,Kenyan Drake,Latavius Murray
Round 9,Matt Breida,Vance McDonald,James White,Larry Fitzgerald,Tarik Cohen,Greg Zuerlin,Emmanuel Sanders,Jared Cook
Round 10,Devonta Freeman,Curtis Samuel,Geronimo Allison,Rashad Penny,Justin Tucker,Mike Thomas,James Washington,Kyler Murray


In [50]:
all_players_19 = draft_19.values.flatten()

In [51]:
all_players_19

array(['Saquon Barkely', 'Ezekiel Elliot', 'Deshaun Watson',
       'Julio Jones', 'Todd Gurley', 'Travis Kelce', 'Antonio Brown',
       'Christian McAffrey', 'Adam Thielan', 'Josh Jacobs',
       'Amari Cooper', 'Chris Carson', 'Zach Ertz', 'Dalvin Cook',
       'LeVeon Bell', 'Keenan Allen', 'David Montgomery', 'Sony Michel',
       'Leornard Fournette', 'Mike Williams', 'Mark Ingram',
       'Melvin Gordon', 'Derrick Henry', 'Kenny Golladay',
       'Brandin Cooks', 'Kerryon Johnson', 'Stefon Diggs', 'Chris Godwin',
       'DeAndre Hopkins', 'Julian Edelman', 'TY Hilton', 'Derius Guice',
       'Tevin Coleman', 'Mike Evans', 'Austin Ekeler', 'Evan Engram',
       'Tyler Boyd', 'Calvin Ridley', 'AJ Green', 'Tyler Lockett',
       'Allen Robinson', 'Robert Woods', 'Tony Pollard',
       'Devin Singletary', 'Christian Kirk', 'Odell Beckham', 'L McCoy',
       'Miles Sanders', 'Aaron Rodgers', 'Robby Anderson',
       'Jarvis Landry', 'Baker Mayfield', 'Duke Johnson',
       'Alshon Je

In [52]:
names_to_fix = check_names(all_players_19, df.PLAYER.values)

In [53]:
len(names_to_fix)

134

In [ ]:
# original_keepers = [p[0] for p in keeper_replacements]

# for p in original_keepers:
#     if p in names_to_fix:
#         print(p)

In [54]:
for p in name_replacements:
    draft_19.replace(to_replace=p[0], value=p[1], inplace=True)

all_players_19 = draft_19.values.flatten()
    
names_to_fix = check_names(all_players_19, df.PLAYER.values)
len(names_to_fix)

91

In [55]:
names_to_fix = remove_fa_k_dst(names_to_fix, unsigned_fa, kickers, defenses)
names_to_fix

array(['nan'], dtype='<U17')

In [56]:
names_to_fix

array(['nan'], dtype='<U17')

In [57]:
draft_19

,Tony,Mike,Joey,Vlad,Dave,Chris,Matt,Ben
Round 1,Saquon Barkley,Ezekiel Elliott,Deshaun Watson,Julio Jones,Todd Gurley,Travis Kelce,Antonio Brown,Christian McCaffrey
Round 2,Adam Thielen,Josh Jacobs,Amari Cooper,Chris Carson,Zach Ertz,Dalvin Cook,Le'Veon Bell,Keenan Allen
Round 3,David Montgomery,Sony Michel,Leonard Fournette,Mike Williams,Mark Ingram II,Melvin Gordon,Derrick Henry,Kenny Golladay
Round 4,Brandin Cooks,Kerryon Johnson,Stefon Diggs,Chris Godwin,DeAndre Hopkins,Julian Edelman,T.Y. Hilton,Derrius Guice
Round 5,Tevin Coleman,Mike Evans,Austin Ekeler,Evan Engram,Tyler Boyd,Calvin Ridley,A.J. Green,Tyler Lockett
Round 6,Allen Robinson,Robert Woods,Tony Pollard,Devin Singletary,Christian Kirk,Odell Beckham Jr.,LeSean McCoy,Miles Sanders
Round 7,Aaron Rodgers,Robby Anderson,Jarvis Landry,Baker Mayfield,Duke Johnson,Alshon Jeffery,Courtland Sutton,Cam Newton
Round 8,Jordan Howard,Marvin Jones,Sammy Watkins,Darrell Henderson,Russell Wilson,Dede Westbrook,Kenyan Drake,Latavius Murray
Round 9,Matt Breida,Vance McDonald,James White,Larry Fitzgerald,Tarik Cohen,Greg Zuerlin,Emmanuel Sanders,Jared Cook
Round 10,Devonta Freeman,Curtis Samuel,Geronimo Allison,Rashaad Penny,Justin Tucker,Mike Thomas,James Washington,Kyler Murray


# Keep Filtering

In [75]:
df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,0.0
2,Ezekiel Elliott,RB,299.6,160.6,3.0,3.0,3.0,0.0
3,Alvin Kamara,RB,296.2,157.2,4.0,5.3,6.0,2.0
4,Dalvin Cook,RB,289.9,150.9,5.0,4.7,4.0,-1.0


In [76]:
keepers_2020

array(['Davante Adams', 'George Kittle', 'D.J. Moore', 'D.J. Chark',
       'JuJu Smith-Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'D.K. Metcalf',
       'Breshad Perriman', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'A.J. Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones II', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliott',
       'Nick Chubb', 'DeVante Parker', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a ', 'Marquise Brown',
       'Russell Wilson', 'Allen Robinson', 'Marlon Mack',
       'Saquon Barkley', 'Miles Sanders', 'David Johnson'], dtype='<U19')

In [77]:
df['KEEPER'] = df.PLAYER.isin(keepers_2020)
df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,0.0,True
2,Ezekiel Elliott,RB,299.6,160.6,3.0,3.0,3.0,0.0,True
3,Alvin Kamara,RB,296.2,157.2,4.0,5.3,6.0,2.0,True
4,Dalvin Cook,RB,289.9,150.9,5.0,4.7,4.0,-1.0,True


## Begin df_no_keep

In [78]:
df_no_keep = df[df["KEEPER"] == False].copy()
df_no_keep

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False
7,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False
12,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False
13,Julio Jones,WR,286.4,102.6,14.0,16.3,18.0,4.0,False
19,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False
21,Todd Gurley,RB,216.5,77.5,22.0,30.7,29.0,7.0,False
22,DeAndre Hopkins,WR,261.2,77.4,23.0,12.0,9.0,-14.0,False
24,Le'Veon Bell,RB,213.8,74.8,25.0,37.3,35.0,10.0,False
27,Chris Carson,RB,207.4,68.4,28.0,32.3,31.0,3.0,False
28,Melvin Gordon,RB,206.8,67.8,29.0,41.0,40.0,11.0,False


## Add Keep Pick Eligibility

In [63]:
keep_el = pd.read_excel('Keeper_Eligibility.xlsx', sheet_name=None)

In [64]:
keep_el['penalty_keeps']

,PLAYER,keep_eligibilty
0,Christian McAffrey,Unkeepable
1,Todd Gurley,Unkeepable
2,Kerryon Johnson,3
3,Mike Evans,1
4,Devonta Freeman,6
5,Davante Adams,11
6,Philip Lindsay,15
7,George Kittle,16
8,Nick Chubb,10
9,DJ Moore,16


In [65]:
keep_el['maintain_position']

,PLAYER,keep_eligibilty
0,Ezekiel Elliott,1
1,Deshaun Watson,1
2,Julio Jones,1
3,Travis Kelce,1
4,Josh Jacobs,1
5,Amari Cooper,1
6,Chris Carson,1
7,Dalvin Cook,1
8,Keenan Allen,1
9,David Montgomery,2


In [66]:
for p in name_replacements:
    keep_el['penalty_keeps'].replace(to_replace=p[0], value=p[1], inplace=True)

In [67]:
for p in name_replacements:
    keep_el['maintain_position'].replace(to_replace=p[0], value=p[1], inplace=True)

In [71]:
all_keeps = keep_el['maintain_position'].merge(keep_el['penalty_keeps'], on='PLAYER', how='outer')


,PLAYER,keep_eligibilty_x,keep_eligibilty_y
0,Ezekiel Elliott,1.0,NaN
1,Deshaun Watson,1.0,NaN
2,Julio Jones,1.0,NaN
3,Travis Kelce,1.0,NaN
4,Josh Jacobs,1.0,NaN
5,Amari Cooper,1.0,NaN
6,Chris Carson,1.0,NaN
7,Dalvin Cook,1.0,NaN
8,Keenan Allen,1.0,NaN
9,David Montgomery,2.0,NaN


In [72]:
all_keeps['KEEPER'] = all_keeps.PLAYER.isin(keepers_2020)

In [74]:
all_keeps = all_keeps[all_keeps['KEEPER'] == False]

In [75]:
all_keeps

,PLAYER,keep_eligibilty_x,keep_eligibilty_y,KEEPER
1,Deshaun Watson,1.0,NaN,False
2,Julio Jones,1.0,NaN,False
3,Travis Kelce,1.0,NaN,False
4,Josh Jacobs,1.0,NaN,False
5,Amari Cooper,1.0,NaN,False
6,Chris Carson,1.0,NaN,False
8,Keenan Allen,1.0,NaN,False
9,David Montgomery,2.0,NaN,False
10,Leonard Fournette,2.0,NaN,False
11,Kenny Golladay,2.0,NaN,False


In [76]:
merge1 = df_no_keep.merge(all_keeps,how='left', on='PLAYER')

In [79]:
merge1

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER_x,keep_eligibilty_x,keep_eligibilty_y,eligible
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False,NaN,Unkeepable,False
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,0.0,False,1.0,NaN,False
2,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False,NaN,NaN,NaN
3,Miles Sanders,RB,249.4,110.4,11.0,13.3,10.0,-1.0,False,6.0,NaN,False
4,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False,1.0,NaN,False
5,Julio Jones,WR,286.4,102.6,14.0,16.3,18.0,4.0,False,1.0,NaN,False
6,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False,1.0,NaN,False
7,Todd Gurley,RB,216.5,77.5,22.0,30.7,29.0,7.0,False,1.0,Unkeepable,False
8,DeAndre Hopkins,WR,261.2,77.4,23.0,12.0,9.0,-14.0,False,1.0,1,False
9,Le'Veon Bell,RB,213.8,74.8,25.0,37.3,35.0,10.0,False,1.0,1,False


In [78]:
merge1 = merge1.rename(columns={'KEEPER_y':'eligible'})

In [237]:
# merge2 = merge1.merge(keep_el['maintain_position'], how='inner', on='PLAYER')

In [70]:
pd.set_option('display.max_rows', None)

In [243]:
# merge2.drop(columns=['keep_eligibilty_x'], inplace=True)

In [80]:
df_final = merge1.copy()

In [81]:
df_final['VALUE_ROUND'] = (df_final['VALUERANK'] // 10) + 1 
df_final

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER_x,keep_eligibilty_x,keep_eligibilty_y,eligible,VALUE_ROUND
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False,NaN,Unkeepable,False,1.0
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,0.0,False,1.0,NaN,False,1.0
2,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False,NaN,NaN,NaN,1.0
3,Miles Sanders,RB,249.4,110.4,11.0,13.3,10.0,-1.0,False,6.0,NaN,False,2.0
4,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False,1.0,NaN,False,2.0
5,Julio Jones,WR,286.4,102.6,14.0,16.3,18.0,4.0,False,1.0,NaN,False,2.0
6,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False,1.0,NaN,False,3.0
7,Todd Gurley,RB,216.5,77.5,22.0,30.7,29.0,7.0,False,1.0,Unkeepable,False,3.0
8,DeAndre Hopkins,WR,261.2,77.4,23.0,12.0,9.0,-14.0,False,1.0,1,False,3.0
9,Le'Veon Bell,RB,213.8,74.8,25.0,37.3,35.0,10.0,False,1.0,1,False,3.0


In [84]:
df_final['keep_score'] = df_final['keep_eligibilty_x'] - df_final['VALUE_ROUND']
df_final

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER_x,keep_eligibilty_x,keep_eligibilty_y,eligible,VALUE_ROUND,keep_score
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False,NaN,Unkeepable,False,1.0,NaN
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,0.0,False,1.0,NaN,False,1.0,0.0
2,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False,NaN,NaN,NaN,1.0,NaN
3,Miles Sanders,RB,249.4,110.4,11.0,13.3,10.0,-1.0,False,6.0,NaN,False,2.0,4.0
4,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False,1.0,NaN,False,2.0,-1.0
5,Julio Jones,WR,286.4,102.6,14.0,16.3,18.0,4.0,False,1.0,NaN,False,2.0,-1.0
6,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False,1.0,NaN,False,3.0,-2.0
7,Todd Gurley,RB,216.5,77.5,22.0,30.7,29.0,7.0,False,1.0,Unkeepable,False,3.0,-2.0
8,DeAndre Hopkins,WR,261.2,77.4,23.0,12.0,9.0,-14.0,False,1.0,1,False,3.0,-2.0
9,Le'Veon Bell,RB,213.8,74.8,25.0,37.3,35.0,10.0,False,1.0,1,False,3.0,-2.0


# Final Keeper Analysis



In [253]:
df_final.sort_values(by='SLEEPERSCORE', ascending=False)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER,keep_eligibilty_y,VALUE_ROUND,keep_score
72,Gerald Everett,TE,84.7,-61.5,189.0,265.0,300.0,111.0,False,18,19.0,-1.0
74,Josh Gordon,WR,112.3,-71.5,197.0,258.3,291.0,94.0,False,18,20.0,-2.0
86,Mike Boone,RB,27.3,-122.3,315.0,330.0,406.0,91.0,False,18,32.0,-14.0
68,Jerick McKinnon,RB,98.0,-51.6,173.0,230.0,246.0,73.0,False,18,18.0,0.0
76,Will Dissly,TE,65.3,-80.9,208.0,251.0,277.0,69.0,False,18,21.0,-3.0
84,Zach Pascal,WR,67.8,-116.0,291.0,295.0,352.0,61.0,False,18,30.0,-12.0
77,Dion Lewis,RB,63.7,-85.9,220.0,234.0,252.0,32.0,False,18,23.0,-5.0
53,Ryan Tannehill,QB,258.2,-16.3,114.0,147.7,146.0,32.0,False,18,12.0,6.0
70,O.J. Howard,TE,89.9,-56.3,181.0,206.3,212.0,31.0,False,18,19.0,-1.0
55,DeSean Jackson,WR,165.4,-18.4,116.0,148.7,147.0,31.0,False,13,12.0,1.0


In [85]:
df_final.to_csv('position_analysis.csv')

# My Team

In [162]:
my_team = ['Saquon Barkley', 'Miles Sanders', 'David Johnson']

In [79]:
df_no_keep.head(30)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False
7,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False
12,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False
13,Julio Jones,WR,286.4,102.6,14.0,16.3,18.0,4.0,False
19,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False
21,Todd Gurley,RB,216.5,77.5,22.0,30.7,29.0,7.0,False
22,DeAndre Hopkins,WR,261.2,77.4,23.0,12.0,9.0,-14.0,False
24,Le'Veon Bell,RB,213.8,74.8,25.0,37.3,35.0,10.0,False
27,Chris Carson,RB,207.4,68.4,28.0,32.3,31.0,3.0,False
28,Melvin Gordon,RB,206.8,67.8,29.0,41.0,40.0,11.0,False


In [80]:
test_df = df_no_keep.copy()

In [81]:
test_df['available'] = True

In [86]:
test_df.loc[test_df['PLAYER'] == 'Christian McCaffrey', ['available']] = True

In [90]:
def avail_updtr(name, df=test_df, status=False):
    if name in df.PLAYER.values:
        df.loc[df['PLAYER'] == name, ['available']] = status
    else:
        print('incorrect name, try again')

In [95]:
avail_updtr('Christian McCaffrey', status=True)

In [94]:
test_df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER,available
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False,False
7,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False,True
12,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False,True
13,Julio Jones,WR,286.4,102.6,14.0,16.3,18.0,4.0,False,True
19,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False,True


In [111]:
def best_avail(position=None, df=test_df, head_n=10):
    if position == None:
        df.sort_values(by="VALUERANK").head(head_n)
    elif position in ['RB','TE','WR','QB']:
        df[df['POS'] == position].sort_values(by="VALUERANK").head(head_n)
    else:
        print('incorrect postion value')

In [117]:
b = best_avail('RB')

In [118]:
b

In [120]:
test_df[test_df['POS'] == 'RB'].sort_values(by="VALUERANK").head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER,available
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False,True
7,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False,True
19,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False,True
21,Todd Gurley,RB,216.5,77.5,22.0,30.7,29.0,7.0,False,True
24,Le'Veon Bell,RB,213.8,74.8,25.0,37.3,35.0,10.0,False,True


In [153]:
test_df.head(20)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER,available
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,0.0,False,True
7,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,17.0,9.0,False,True
12,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False,True
13,Julio Jones,WR,286.4,102.6,14.0,16.3,18.0,4.0,False,True
19,Josh Jacobs,RB,222.6,83.6,20.0,14.3,12.0,-8.0,False,True
21,Todd Gurley,RB,216.5,77.5,22.0,30.7,29.0,7.0,False,True
22,DeAndre Hopkins,WR,261.2,77.4,23.0,12.0,9.0,-14.0,False,True
24,Le'Veon Bell,RB,213.8,74.8,25.0,37.3,35.0,10.0,False,True
27,Chris Carson,RB,207.4,68.4,28.0,32.3,31.0,3.0,False,True
28,Melvin Gordon,RB,206.8,67.8,29.0,41.0,40.0,11.0,False,True


In [163]:
my_team

['Saquon Barkley', 'Miles Sanders', 'David Johnson']

In [164]:
my_test_team = ['Saquon Barkley', 'Miles Sanders', 'David Johnson', 
                'Dak Prescott','Keenan Allen','Adam Thielen',
                'Amari Cooper','Zach Ertz','Travis Kelce']

In [165]:
my_test_team

['Saquon Barkley',
 'Miles Sanders',
 'David Johnson',
 'Dak Prescott',
 'Keenan Allen',
 'Adam Thielen',
 'Amari Cooper',
 'Zach Ertz',
 'Travis Kelce']

In [122]:
team_roster = {
    'QB':None,
    'RB1':None,
    'RB2':None,
    'WR1':None,
    'WR2':None,
    'FLEX1':None,
    'FLEX2':None,
    'TE':None,
    'K':None,
    'DST':None,
    'Bench1':None,
    'Bench2':None,
    'Bench3':None,
    'Bench4':None,
    'Bench5':None,
    'Bench6':None,
    'Bench7':None,
    'Bench8':None,  
}

In [ ]:
# determine player position
# top player in position = Pos1
# if RB or WR pos - second player in position = Pos2 
# all other players go to "yet to place pool"
# after players placed - top 2 WR/RB/TE become Flex1/Flex2
# finally all players are on bench

In [166]:
my_tm_df = df[df['PLAYER'].isin(my_test_team)]

In [167]:
my_tm_df

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,0.0,True
10,Miles Sanders,RB,249.4,110.4,11.0,13.3,10.0,-1.0,True
12,Travis Kelce,TE,252.7,106.5,13.0,18.0,20.0,7.0,False
25,David Johnson,RB,211.5,72.5,26.0,38.3,38.0,12.0,True
30,Zach Ertz,TE,211.1,64.9,31.0,38.0,37.0,6.0,False
37,Amari Cooper,WR,230.8,47.0,38.0,34.0,34.0,-4.0,False
39,Adam Thielen,WR,229.9,46.1,40.0,33.3,33.0,-7.0,False
40,Keenan Allen,WR,228.4,44.6,41.0,49.3,48.0,7.0,False
41,Dak Prescott,QB,318.4,43.9,42.5,53.7,52.0,9.5,False


In [200]:
def make_roster(team_members, df=my_tm_df, roster=team_roster):
    #place_later = []
    #place RBs 
    rbs = df[df['POS'] == 'RB'].sort_values(by='FPTS', ascending=False)
   
    team_roster['RB1'] = (rbs.iloc[0,0], rbs.iloc[0,2])
    
    team_roster['RB2'] = (rbs.iloc[1,0], rbs.iloc[1,2])
    
#     place_later = rbs.iloc[2:,0].values
    
    #place WRs 
    wrs = df[df['POS'] == 'WR'].sort_values(by='FPTS', ascending=False)
   
    team_roster['WR1'] = (wrs.iloc[0,0], wrs.iloc[0,2])
    
    team_roster['WR2'] = (wrs.iloc[1,0], wrs.iloc[1,2])
    
#     place_later = place_later + (wrs.iloc[2:,0].values)
    
    #place QBs 
    qbs = df[df['POS'] == 'QB'].sort_values(by='FPTS', ascending=False)
   
    team_roster['QB'] = (qbs.iloc[0,0], qbs.iloc[0,2])
    
#     place_later.append(qbs.iloc[1:,0].values)
#     place_later = place_later + (qbs.iloc[1:,0].values)
    
    #place TEs 
    tes = df[df['POS'] == 'TE'].sort_values(by='FPTS', ascending=False)
   
    team_roster['TE'] = (tes.iloc[0,0], tes.iloc[0,2])
    
#     place_later.append(tes.iloc[1:,0].values)
#     place_later = place_later + (tes.iloc[1:,0].values)
    
    place_later = (rbs.iloc[2:,0].values)+(wrs.iloc[2:,0].values)+(qbs.iloc[1:,0].values)+(tes.iloc[1:,0].values)
    display(place_later)
    #place Flex
#     flexs = df[df['PLAYER'].isin(place_later)].sort_values(by='FPTS',
#                                                            ascending=False)
# #     eligible_pos = ['RB','WR','TE']
    
# #     flexs = flexs[flexs['POS'].isin(eligible_pos)]
    
#     team_roster['FLEX1'] = (flexs.iloc[0,0], flexs.iloc[0,2])
    
#     team_roster['FLEX2'] = (flexs.iloc[1,0], flexs.iloc[1,2])

In [148]:
my_tm_df.iloc[2:,0].values

array(['David Johnson'], dtype=object)

In [201]:
make_roster(my_team)

array([], dtype=object)

In [202]:
team_roster

{'QB': ('Dak Prescott', 318.4),
 'RB1': ('Saquon Barkley', 310.7),
 'RB2': ('Miles Sanders', 249.39999999999998),
 'WR1': ('Amari Cooper', 230.8),
 'WR2': ('Adam Thielen', 229.9),
 'FLEX1': None,
 'FLEX2': None,
 'TE': ('Travis Kelce', 252.7),
 'K': None,
 'DST': None,
 'Bench1': None,
 'Bench2': None,
 'Bench3': None,
 'Bench4': None,
 'Bench5': None,
 'Bench6': None,
 'Bench7': None,
 'Bench8': None}

# Graveyard

In [ ]:
# keeper_history.replace(to_replace='DJ Moore', value='D.J. Moore', inplace=True)

In [ ]:
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [ ]:
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Chark' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='DJ Chark', value='D.J. Chark', inplace=True)

In [ ]:
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [ ]:
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Schuster' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='Juju Smith Schuster', value='JuJu Smith-Schuster', inplace=True)

In [ ]:
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [ ]:
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Metcalf' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='DK Metcalf', value='D.K. Metcalf', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Perriman' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='Brashad Perrimen', value='Breshad Perriman', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Brown' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='AJ Brown', value='A.J. Brown', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Jones' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='Ronald Jones', value='Ronald Jones II', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Ezekiel' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='Ezekiel Elliot ', value='Ezekiel Elliott', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [ ]:
# for p in df.PLAYER.values:
#     if 'Parker' in p:
#         print(p)

In [ ]:
# keeper_history.replace(to_replace='Devante Parker ', value='DeVante Parker', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix